In [5]:
import numpy as np
import sympy as sp
import math
from sympy import ntheory
from sympy.ntheory import continued_fraction_periodic

In [64]:
def classRepresentativesTrace(trace): # gives quadratic forms of given trace
    bound = math.floor(trace**2-4)
    quadraticForms = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
        i = i + 1
    return [Q for Q in quadraticForms if np.trace(quadToMat(Q))==np.round(trace)]

def classRepresentativesBounded(length): # gives quadratic forms of length bounded by a quantity
    trace = np.round(2*np.cosh(length/2))
    bound = math.floor(trace**2-4)
    quadraticForms = []
    for D in range(2,bound+1):
        if isASquare(D):
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
    return quadraticForms

def representatives(D):
    reps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                reps.append(Q)
            else:
                continue
    return reps

def permutationsOf(a):
    perms = []
    if len(a)%2==0:
        for n in range(0,len(a),2):
            perms.append(a[n:]+a[:n])
    else:
        for n in range(0,len(a)):
            perms.append(a[n:]+a[:n])
    return perms

def disc(Q):
    return Q[1]**2-4*Q[0]*Q[2]

def areEquivalent(Q,W):
    return ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] in permutationsOf(ntheory.continued_fraction_periodic(-W[1], 2*W[0], disc(W))[0])

def equalsQuad(L,M):
    if len(L)!=len(M):
        return False
    for l in L:
        counter = 0
        for m in M:
            if areEquivalent(m,l):
                M.remove(m)
                counter+=1
        if counter !=1:
            return False
    return True

def classNum(D):
    return len(representatives(D))

def tr(Q):
    return np.trace(quadToMat(Q))

In [17]:
def isASquare(n):
    for i in range(2,n):
        if n==i**2:
            return True
    return False
def gcd(a,b,c):
    return np.gcd(a,np.gcd(b,c))
def matgcd(mat):
    return gcd(mat[1,0],mat[1,1]-mat[0,0],mat[0,1])

In [18]:
def quadToMat(Q):
    mat = np.identity(2)
    length = 0
    for x in ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [47]:
def a(t):
    print([(t,tr(Q),Q,disc(Q)) for Q in classRepresentatives(2*np.arccosh(t/2))])
[a(t) for t in range(3,20)]

[(3, 3, [1, -1, -1], 5)]
[(4, 4, [1, -2, -2], 12), (4, 4, [2, -2, -1], 12)]
[(5, 5, [1, -3, -3], 21), (5, 5, [3, -3, -1], 21)]
[(6, 6, [1, -4, -4], 32), (6, 6, [4, -4, -1], 32), (6, 6, [1, -2, -1], 8)]
[(7, 7, [1, -5, -5], 45), (7, 7, [5, -5, -1], 45), (7, 3, [1, -1, -1], 5)]
[(8, 8, [1, -6, -6], 60), (8, 8, [2, -6, -3], 60), (8, 8, [3, -6, -2], 60), (8, 8, [6, -6, -1], 60)]
[(9, 9, [1, -7, -7], 77), (9, 9, [7, -7, -1], 77)]
[(10, 10, [4, -4, -5], 96), (10, 10, [5, -4, -4], 96), (10, 10, [1, -8, -8], 96), (10, 10, [8, -8, -1], 96), (10, 10, [1, -4, -2], 24), (10, 10, [2, -4, -1], 24)]
[(11, 11, [1, -9, -9], 117), (11, 11, [9, -9, -1], 117), (11, 11, [1, -3, -1], 13)]
[(12, 12, [1, -10, -10], 140), (12, 12, [2, -10, -5], 140), (12, 12, [5, -10, -2], 140), (12, 12, [10, -10, -1], 140)]
[(13, 13, [5, -5, -7], 165), (13, 13, [7, -5, -5], 165), (13, 13, [1, -11, -11], 165), (13, 13, [11, -11, -1], 165)]
[(14, 14, [1, -12, -12], 192), (14, 14, [3, -12, -4], 192), (14, 14, [4, -12, -3], 192),

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [57]:
quads = classRepresentativesBounded(5)
for quad in quads:
    print(quad,disc(quad))

[1, -1, -1] 5
[1, -2, -1] 8
[1, -2, -2] 12
[2, -2, -1] 12
[1, -3, -1] 13
[2, -1, -2] 17
[1, -4, -1] 20
[1, -3, -3] 21
[3, -3, -1] 21
[1, -4, -2] 24
[2, -4, -1] 24
[2, -2, -3] 28
[3, -2, -2] 28
[1, -5, -1] 29
[1, -4, -4] 32
[4, -4, -1] 32
[2, -3, -3] 33
[3, -3, -2] 33
[3, -1, -3] 37
[3, -2, -3] 40
[1, -6, -1] 40
[2, -3, -4] 41
[1, -6, -2] 44
[2, -6, -1] 44
[1, -5, -5] 45
[5, -5, -1] 45
[1, -6, -3] 48
[3, -6, -1] 48
[3, -2, -4] 52
[1, -7, -1] 53
[2, -4, -5] 56
[5, -4, -2] 56
[3, -3, -4] 57
[4, -3, -3] 57
[1, -6, -6] 60
[2, -6, -3] 60
[3, -6, -2] 60
[6, -6, -1] 60
[3, -5, -3] 61
[4, -1, -4] 65
[2, -5, -5] 65
[1, -8, -1] 68
[3, -3, -5] 69
[5, -3, -3] 69
[1, -8, -2] 72
[2, -8, -1] 72
[4, -3, -4] 73
[3, -4, -5] 76
[5, -4, -3] 76
[1, -7, -7] 77
[7, -7, -1] 77
[1, -8, -4] 80
[4, -8, -1] 80
[4, -2, -5] 84
[5, -2, -4] 84
[3, -5, -5] 85
[1, -9, -1] 85
[3, -4, -6] 88
[6, -4, -3] 88
[4, -3, -5] 89
[2, -6, -7] 92
[7, -6, -2] 92
[1, -9, -3] 93
[3, -9, -1] 93
[4, -4, -5] 96
[5, -4, -4] 96
[1, -8, -8] 

In [59]:
Q = [1, -3, -3]
ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]

[3, 1]

In [60]:
Q = [3, -3, -1]
ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]

[1, 3]

In [80]:
for D in range(2,1000):
    if isASquare(D):
        continue
    if (D % 4 == 0) or (D % 4 == 1): 
        print("h0(" + str(D) + ") = " + str(classNum(D)))

h0(5) = 1
h0(8) = 1
h0(12) = 2
h0(13) = 1
h0(17) = 1
h0(20) = 1
h0(21) = 2
h0(24) = 2
h0(28) = 2
h0(29) = 1
h0(32) = 2
h0(33) = 2
h0(37) = 1
h0(40) = 2
h0(41) = 1
h0(44) = 2
h0(45) = 2
h0(48) = 2
h0(52) = 1
h0(53) = 1
h0(56) = 2
h0(57) = 2
h0(60) = 4
h0(61) = 1
h0(65) = 2
h0(68) = 1
h0(69) = 2
h0(72) = 2
h0(73) = 1
h0(76) = 2
h0(77) = 2
h0(80) = 2
h0(84) = 2
h0(85) = 2
h0(88) = 2
h0(89) = 1
h0(92) = 2
h0(93) = 2
h0(96) = 4
h0(97) = 1
h0(101) = 1
h0(104) = 2
h0(105) = 4
h0(108) = 2
h0(109) = 1
h0(112) = 2
h0(113) = 1
h0(116) = 1
h0(117) = 2
h0(120) = 4
h0(124) = 2
h0(125) = 1
h0(128) = 2
h0(129) = 2
h0(132) = 2
h0(133) = 2
h0(136) = 4
h0(137) = 1
h0(140) = 4
h0(141) = 2
h0(145) = 4
h0(148) = 3
h0(149) = 1
h0(152) = 2
h0(153) = 2
h0(156) = 4
h0(157) = 1
h0(160) = 4
h0(161) = 2
h0(164) = 1
h0(165) = 4
h0(168) = 4
h0(172) = 2
h0(173) = 1
h0(176) = 2
h0(177) = 2
h0(180) = 2
h0(181) = 1
h0(184) = 2
h0(185) = 2
h0(188) = 2
h0(189) = 2
h0(192) = 4
h0(193) = 1
h0(197) = 1
h0(200) = 2
h0(201) = 